In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from polygon_streaming_api import run_stream_ticks
from polygon_rest_api import get_grouped_daily, get_stock_ticks
from polygon_backfill import *
import bars

import pandas_bokeh
# pandas_bokeh.output_notebook()
pandas_bokeh.output_file('bokeh_output.html')
# pd.set_option('plotting.backend', 'pandas_bokeh')

# from IPython.core.debugger import set_trace; set_trace()

In [2]:
symbol='DUST'
start_date='2020-06-01'
end_date='2020-06-20'
result_path="/Users/bobcolner/QuantClarity/data"
date_partition='hive'
tick_type='trades'
formats=['feather', 'parquet']
skip=True

In [268]:
def read_market_daily(result_path:str) -> pd.DataFrame:    
    df = read_matching_files(glob_string=result_path+'/market_daily/*.feather', reader=pd.read_feather)
    df = df.set_index(pd.to_datetime(df.date), drop=True)
    df = df.drop(columns=['date'])
    df = find_compleat_symbols(df, compleat_only=True)
    df = df.sort_index()
    return df

In [309]:
def plot_daily_symbols(df:pd.DataFrame, symbols=['SPY', 'QQQ'], metric='vwap') -> pd.DataFrame:
    fdf = df[['symbol', metric]][df.symbol.isin(symbols)]
    pdf = fdf.pivot(columns='symbol', values=metric)
    pdf.plot_bokeh.line(sizing_mode="scale_height", rangetool=True, title=str(symbols), ylabel=metric+' [$]', number_format="1.00 $")
    return pdf

In [ ]:
df = read_market_daily(result_path)

In [319]:
dfp = df.pivot(columns='symbol', values='vwap')

In [330]:
dfp

symbol,A,AA,AADR,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZIXI,ZN,ZNGA,ZROZ,ZSL,ZTO,ZTR,ZTS,ZUMZ,ZYNE
date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,46.339901,28.549700,41.310299,46.542900,31.872400,23.3249,33.311100,170.246094,115.612701,43.005299,...,4.9316,1.3706,2.5988,108.643700,37.576698,12.213300,11.8925,53.718899,21.961599,17.002199
2017-01-04,47.145901,29.860800,41.789200,46.828701,32.497898,23.4492,33.912800,172.152802,116.131798,43.526901,...,5.0054,1.4240,2.6877,109.557602,36.300499,12.247800,11.8441,53.784199,22.744400,18.899099
2017-01-05,46.546902,30.580601,42.412498,46.163200,31.321100,22.8790,33.012001,171.628799,116.461098,43.592300,...,4.9380,1.4428,2.6608,111.474998,35.486500,12.908600,11.3374,53.854198,20.775999,19.341900
2017-01-06,47.633301,30.667801,42.308601,46.203499,31.079599,22.6467,32.860199,170.364594,117.733398,43.806599,...,4.8749,1.3683,2.6993,111.265198,36.160599,13.295100,11.2718,54.259399,20.161100,18.768000
2017-01-09,48.205200,29.730301,42.533298,46.905499,31.202600,22.2852,32.753601,169.427994,118.979897,43.287800,...,4.9105,1.3324,2.6957,112.236900,35.558701,13.034600,11.2641,54.044800,20.008101,18.141001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-15,86.389801,11.191200,49.515499,16.274799,43.352402,8.9447,52.280399,136.399200,339.512604,29.961000,...,6.5233,0.3171,9.1217,172.735199,22.439699,36.029301,8.3379,132.790497,26.674601,4.992000
2020-06-16,87.661797,11.750800,50.902000,17.385500,45.136002,9.2428,54.006802,142.240005,350.871490,31.335800,...,7.0122,0.3325,9.0791,166.675293,21.690901,36.250198,8.5717,136.318802,28.170300,5.402200
2020-06-17,87.986603,11.632700,51.446800,16.664801,44.614601,10.0176,53.367802,147.519897,353.714386,30.064199,...,7.1544,0.3289,9.3094,167.283401,21.512400,37.024601,8.4927,136.869598,27.316000,5.477200


In [ ]:
plot_daily_symbols(df)

In [ ]:
backfill_ticks(symbol, start_date, end_date, result_path, date_partition, tick_type, formats, skip)

In [ ]:
# https://arrow.apache.org/docs/python/dataset.html
import pyarrow.dataset as ds

dataset = ds.dataset(
    source=result_path+'/ticks/trades/feather',
    format="feather", 
    partitioning="hive"
)

In [ ]:
# df = dataset.to_table(columns=['tick_epoch', 'price', 'size'], filter=ds.field("size") > 999).to_pandas()
df = dataset.to_table(filter=ds.field("symbol") == 'GLD').to_pandas()
df.info()


In [ ]:
# https://arrow.apache.org/docs/python/dataset.html
import pyarrow.dataset as ds

dataset = ds.dataset(
    source=result_path+'/ticks/trades/feather', 
    format="feather", 
    partitioning="hive"
)

df = dataset.to_table(columns=['tick_epoch', 'price', 'size'], filter=ds.field("size") > 999).to_pandas()

sdf = dataset.to_batches()
f = next(sdf)
f.to_pandas()

In [ ]:

def compound_interest(principle, rate, time): 
    # Calculates compound interest  
    total_return = principle * (pow((1 + rate / 100), time)) 
    print("Total Return:", total_return)
    return round(total_return / principle, 1) * 100


principle=100
compound_interest(principle, 0.3, 250)


In [ ]:
import gcsfs

fs = gcsfs.GCSFileSystem()

path = 'gcs://emerald-skill-datascience/imported/automl_predictions/prediction-premium_d30_train_sample-2020-03-16T22:15:53.212Z'
fs.ls(path)
fo = fs.open(path)

{'cache_storage': '/User/bobcolner/QuantClarity/data/tmp'}

fo = fs.open(
    path='filecache::'+path+'/tables_1.csv',
    mode='r',
    cache_options={'cache_storage':'/User/bobcolner/QuantClarity/data/tmp'}
)

fo.readline()    

In [ ]:
import fsspec

fso = fsspec.open(
    urlpath='filecache::'+path+'/tables_1.csv',
    filecache={'cache_storage':'/User/bobcolner/QuantClarity/data/tmp'}
)
